# Logarithmic Horns

This Jupyter notebook should allow you to experiment with the plotting of logarithmic horns using [Plotly](https://plot.ly/python/). Have a read through everything before you start running things.

---

**If running any of the following code produces the error:** "`IOPub data rate exceeded.`", you may need to run the notebook with an additional setting.

If you open Jupyter Notebooks from Anaconda:

* At the Anaconda Prompt, type:

  `jupyter notebook --generate-config`
  
  
* In your home folder (something like `C:\Users\brian`), find the `.jupyter\jupyter_notebook_config.py` file and open in your favourite text editor. Notepad will do.


* Find the line that reads

  `#c.NotebookApp.iopub_data_rate_limit = ...`
  
  and make the limit significantly higher, also removing the comment:
  
  `c.NotebookApp.iopub_data_rate_limit = 1000000000`
  
  
* Save and close the file
* Restart the Jupyter Notebook.

Or if you run notebooks from the command line, try:

`jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000.0`

---

Begin by importing useful libraries and initialising things for a Jupyter Notebook:

In [ ]:
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode()

### Plotting a horn

We now define a function `draw_log_horn()` that, given the horn's parameters, will create a plot.
The two key parameters to explore are $b$ and $w_0$ and so these have to be given each time the function is called, others can be left with sensible default values at least to start with. These default values means we can call the function without providing them. Some, like $\alpha$ only make sense in specific ranges: $0 \le \alpha \le 2\pi$.

In Python if we want to use the constant $\pi = 3.14159265..$ we can either define it ourselves or, better, use a constant from a library like NumPy; in this case since we've imported NumPy as `np`, we can get the constant $\pi$ using `np.pi`.

We name our variables after the Greek letters and use an underscore to denote subscripts: $\alpha$ becomes `alpha`, $\theta$ becomes `theta`, $w_0$ becomes `w_0`.

One parameter of the function we're going to define has nothing to do with the maths itself, but is instead part of our computational model: `divisions_per_radian`. In order to make a 3D plot, we need to sample the surface at many points and this allows us to adjust how many. A real function has a value at every point in space, but computers cannot cope with an infinite number of values, so we limit how many samples we use. Too few, and the plot will look rough and unlike the real function; too many and our code will never be able to work out all of the values required and may crash.

The star (or asterix, `*`) as the first argument of the function definition means that arguments to this function have to be passed by name; so `draw_log_horn(0.2, 0.5)` will not work, and we have to be clear what number is what: `draw_log_horn(b=0.2, w_0=0.5)`.

In [ ]:
def draw_log_horn(*, b, w_0, alpha_min=0, alpha_max=2*np.pi, theta_min=-6*np.pi, theta_max=0, divisions_per_radian=8):

    # Check the inputs are sensible:

    if alpha_min > alpha_max:
        print("Error: alpha_min must be smaller than alpha_max!")
        return
    if theta_min > theta_max:
        print("Error: theta_min must be smaller than theta_max!")
        return
    if not(divisions_per_radian > 0 and isinstance(divisions_per_radian, int)):
        print("Error: the number of divisions per radian must be a positive integer!")
        return

    # Set up the grid of points we're going to sample at to create a surface:

    alpha_range = alpha_max - alpha_min
    alpha_array = np.linspace(alpha_min, alpha_max, int(alpha_range * divisions_per_radian))
    
    theta_range = theta_max - theta_min
    theta_array = np.linspace(theta_min, theta_max, int(theta_range * divisions_per_radian))
    
    alpha_grid, theta_grid = np.meshgrid(alpha_array, theta_array)

    # Work out the values of r and w at each point:

    r_0 = 1
    r = r_0 * np.exp(b * theta_grid)  # underlying log spiral:     r = r_0 * exp(b*theta)
    w = w_0 * np.exp(b * theta_grid)  # circular sectional radius: w = w_0 * exp(b*theta)

    # Convert from angular coordinates to linear coordinates (note r and w are arrays here):

    x = (r + w * np.cos(alpha_grid)) * np.cos(theta_grid)  # x = (r + w*cos(alpha))*cos(theta)
    y = (r + w * np.cos(alpha_grid)) * np.sin(theta_grid)  # y = (r + w*cos(alpha))*sin(theta)
    z = w * np.sin(alpha_grid)                             # z = w*sin(alpha)

    # Configure the Plotly plot:

    surface = go.Surface(x=x, y=y, z=z)
    data = [surface]
    layout = go.Layout(
        height=800,
        scene=dict(
            xaxis=dict(
                gridcolor='rgb(0, 0, 255)',
                zerolinecolor='rgb(255, 0, 0)',
                showbackground=False,
                backgroundcolor='rgb(230, 230, 230)'
            ),
            yaxis=dict(
                gridcolor='rgb(0, 0, 255)',
                zerolinecolor='rgb(255, 0, 0)',
                showbackground=False,
                backgroundcolor='rgb(230, 230, 230)'
            ),
            zaxis=dict(
                gridcolor='rgb(0, 0, 255)',
                zerolinecolor='rgb(255, 0, 0)',
                showbackground=False,
                backgroundcolor='rgb(230, 230, 230)'
            )
        )
    )
    fig = go.Figure(data=data, layout=layout)

    # Plot it:

    py.iplot(fig)

### Exploring the parameters

We start off with the values $b = 0.2$ and $w_0 = \frac{1}{2}r_0 = 0.5$, which produce a log horn like that on the cover of the book. These are given as arguments when we call the function we've just defined. Some of the function parameters have default values which means we don't have to specify them every time, but can provide them to override the defaults if we want to see what happens. So we'll start with `draw_log_horn(b=0.2, w_0=0.5)`.

When you're running this notebook yourself for the first time, go to the menu at the top and click "Cell" -> "Run All". Look at the plot this produces below (you may need to mouse over it to see it initially). To explore, you can drag the view of the plot around and zoom in with the mouse wheel. Then you can change the arguments of the function in the cell below and press `Ctrl-Enter` to re-run that cell and update the plot.

Things you might like to try:

* How does changing the value of $w_0$ compared to the fixed value of $r_0 = 1$ affect the spiral? Try $w_0 \ll r_0$,  $w_0 = r_0$  or  $w_0 > r_0$.
* What happens if $\alpha$ does not start at $0$? Try increasing the value of `alpha_min` when calling `draw_log_horn()`.
* We've plotted $\theta$ in the range $-6\pi \le \theta \le 0$ (i.e. `theta_min = -6*np.pi` and `theta_max = 0`).
  What happens to the shape of the plot if you use the range $0 \le \theta \le 6\pi$, say, or even $-4\pi \le \theta \le 2\pi$? 
  What property of the log horn does this demonstrate?

In [ ]:
draw_log_horn(b=0.2, w_0=0.5)